In [1]:
import os
import sys
import pandas as pd
import googletrans
from googletrans import Translator
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans 
import numpy as np
from sklearn.model_selection import GridSearchCV


In [2]:
# Constants
ROOT_PATH = os.path.abspath(os.getcwd())
SEED = 170

# bycode 2018 excel
BYCODE = "bycode2018"

# index 2018 excel
INDEX = "index 2018"
NATURAL = "Natural Area"
DISTRICT = "District"
RELIGION = "Religion"
SETTLEMENT_TYPE = "Settlement Type"


# elections data
CALPI = "calpi"
SETTELMENT = "settelments"


In [3]:
def inner_join(df_1,df_2,column_name):
    """makes inner-join between dataframes on the specified column"""
    return pd.merge(left=df_1, right=df_2, left_on= column_name, right_on=column_name)

def get_index(sheet):
    "returns dataframe of sheet from index 2018 excel"
    path = os.path.join(ROOT_PATH,INDEX+".xlsx")
    return pd.read_excel(path, sheet_name=sheet)

def get_bycode():
    """returns dataframe for bycode excel"""
    path = os.path.join(ROOT_PATH,BYCODE+".xlsx")
    return pd.read_excel(path)

def get_data(agg_type, num):
    """returns dataframe of the requested .xlsx""" 
    if type(num) is not str:
        num = str(num)
        
    path = os.path.join(ROOT_PATH,num+agg_type+".xlsx")
    return pd.read_excel(path,dtype=object)

In [4]:
def remove_small_parties(df,threshold):
    """remove parties that didnt pass the threshold"""
    
    for column in df.columns[7:]:
            if df[column].sum() < threshold:
                   df = df.drop(column,axis=1)
    return df

In [7]:
def total_voters(df):
    "ploting a barchar from dataframe"
    dict={}
    for c in df.columns[7:]:
        dict[c]=(df[c].sum()/df['valid votes'].sum())*100
    plt.figure(figsize=(20,10))
    plt.bar(range(len(dict)), list(dict.values()), align='center')
    plt.ylabel('Voter turnout')
    plt.xlabel('Parties')
    plt.xticks(range(len(dict)), list(dict.keys()))
    plt.show()

In [8]:
def unite_parties (df):
    """ unites small parties to factions
    
    We think those factions represent the israeli society
    """
    
    d={'United Torah Judaism':'Haredi','Shas':'Haredi', 'Avoda':'Left','Meretz':'Left',
       'Consolidation of right-wing parties':'Right','Kolano':'Right','Israel is our Home':'Right','New Right':'Right',
        'UAL-Balad':'Arab','Hadash':'Arab' ,'Gesher Avoda':'Left','Joint list':'Arab','right':'Right'
        ,'Avoda-Meretz-Gesher':'Left'}
    
    faction=['Haredi','Right','Arab','Left']
    for f in faction:
        df.insert(len(df.columns),f ,0)
    for c in  df.columns[7:]:
        if c in d:
            s=df[d[c]]+df[c]
            df[d[c]]=s
   
    for c in d.keys():
        if c in df.columns:
              df=df.drop(c,axis=1)
    return df

In [9]:
def normalize_to_voting_ratios(df):
    """normalizing the votes according to proportion of votes per party"""
    
    for i,r in df.iterrows():
        for c in df.columns[7:]:
            x = r[c]/r['valid votes']
            df.at[i , c]=x
    
    colms_to_remove = ["name","Holders of voting rights","Voters","Disqualified","valid votes","committee code"]
    df = df.drop(labels = colms_to_remove,axis=1) 
    return df

In [10]:
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ 
    Returns the angle in radians between vectors 'v1' and 'v2'
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

def prepare_vectors(df):
    """calculates the distance and the angle of each vector from the base_vector"""
    
    matrix = [] 
    vector_base = np.zeros(df.shape[1])
    vector_base.fill(1)

    for row in df:
        dist = np.linalg.norm(row)
        angle = angle_between(vector_base,row)
        matrix.append([dist,angle])
        
    return np.array(matrix)
    

In [9]:
def add_most_voted_colm(df):
    """ adds a column of labels for the most voted faction """
    temp = df.copy()
    temp.drop(labels = ["code"],axis=1,inplace = True)
    
    for c in temp.columns:
        temp[c] = pd.to_numeric(temp[c])
    
    colm = temp.idxmax(axis=1)
    df["chosen"] = colm
    return df
        

In [26]:
df_21 = get_data(SETTELMENT,21)
df_21 = remove_small_parties(df_21,135720)
df_21 = unite_parties(df_21)
df_21 = normalize_to_voting_ratios(df_21)
df_21 = add_most_voted_colm(df_21)

In [11]:
from statsmodels.graphics.gofplots import qqplot

s = df['chosen'].value_counts()
performance=  np.array(s.values)
qqplot(performance)

plt.show()

NameError: name 'df' is not defined

In [ ]:

vectors = df_21[df_21.columns.difference(['code','chosen'])]
vectors = prepare_vectors(vectors.values)

In [ ]:
param = {
    "n_clusters":[3,6],
    "init":["k-means++", "random"],
    "n_init": [10],
    "max_iter":[300],
    "precompute_distances":[False,True],
    "random_state":[SEED],
    "algorithm" : ["auto", "full", "elkan"]

}
gs = GridSearchCV(KMeans(), param,  refit=True, cv=[(slice(None), slice(None))], verbose=1)
gs.fit(vectors)
y = model.predict(vectors)
df_21['Cluster Class'] = pd.Series(y, index=df_21.index)

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(221)
plt.scatter(vectors[:, 0], vectors[:, 1], c=y)
plt.title("Incorrect Number of Blobs")

In [ ]:
# Prepare data
x_var = 'Cluster Class'
groupby_var = 'chosen'
df_agg = df_21.loc[:, [x_var, groupby_var]].groupby(groupby_var)
vals = [df_21[x_var].values.tolist() for i, df_21 in df_agg]

# Draw
plt.figure(figsize=(16,9), dpi= 80)
colors = [plt.cm.Spectral(i/float(len(vals)-1)) for i in range(len(vals))]
n, bins, patches = plt.hist(vals, df_21[x_var].unique().__len__(), stacked=False, density=False, color=colors[:len(vals)])

# Decoration
plt.legend({group:col for group, col in zip(np.unique(df_21[groupby_var]).tolist(), colors[:len(vals)])})
plt.title(f"Stacked Histogram of ${x_var}$ colored by ${groupby_var}$", fontsize=22)
plt.xlabel(x_var)
plt.ylabel("Frequency")
plt.ylim(0, 300)
plt.xticks(ticks=bins, labels=np.unique(df_21[x_var]).tolist(), rotation=90, horizontalalignment='left')
plt.show()

In [ ]:
bycode = get_bycode()
index = get_index(DISTRICT)
df = inner_join(df_21,bycode,"code")
df = inner_join(df,index,"District code")
df_211 = df[["Cluster Class","District code"]]

In [11]:
# Prepare data
x_var = 'Cluster Class'
groupby_var = 'Religion code'
df_agg = df_21.loc[:, [x_var, groupby_var]].groupby(groupby_var)
vals = [df_21[x_var].values.tolist() for i, df_21 in df_agg]
# Draw
plt.figure(figsize=(16,9), dpi= 80)
colors = [plt.cm.Spectral(i/float(len(vals)-1)) for i in range(len(vals))]
n, bins, patches = plt.hist(vals, df_21[x_var].unique().__len__(), stacked=False, density=False, color=colors[:len(vals)])

# Decoration
plt.legend({group:col for group, col in zip(np.unique(df_21[groupby_var]).tolist(), colors[:len(vals)])})
plt.title(f"Stacked Histogram of ${x_var}$ colored by ${groupby_var}$", fontsize=22)
plt.xlabel(x_var)
plt.ylabel("Frequency")
plt.ylim(0, 150)
plt.xticks(ticks=bins, labels=np.unique(df_21[x_var]).tolist(), rotation=90, horizontalalignment='left')
plt.show()

NameError: name 'df_21' is not defined

In [ ]:
import GaussianMixture
clf = mixture.GaussianMixture(n_components=2, covariance_type='full')
clf.fit(X_train)

In [12]:
def getdataRegress(num , RF ,n=None):
    
        df = get_data(SETTELMENT,num)
        df = remove_small_parties(df,135720)
        df = unite_parties(df)
        if RF:
            return df[["committee code","Voters","Disqualified","Holders of voting rights","Likud",
                   "Blue and white","Haredi","Right","Arab","Left","valid votes"]]
        if n == 1:
            return df[["committee code","Voters","Disqualified","Holders of voting rights","Likud",
                   "Haredi","Right","Arab","Left","valid votes","Blue and white"]]
        elif n == 2:
            return df[["committee code","Voters","Disqualified","Holders of voting rights",
                   "Blue and white","Haredi","Right","Arab","Left","valid votes","Likud"]]
        else:
            return df[["committee code","Voters","Disqualified","Holders of voting rights","Likud",
                   "Blue and white","Haredi","Right","Left","valid votes","Arab"]]

In [13]:
from sklearn.ensemble import  RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn import  metrics
param_grid = {
    'n_estimators': [100, 150, 200, 250],  # The number of trees in the forest.
    'max_depth': [None, 50, 60, 70],  # The maximum depth of the tree.
    'max_features': ['sqrt', None],  # he number of features to consider when looking for the best split
    'min_samples_split': [2, 5, 10],  # The minimum number of samples required to split an internal node
    'bootstrap': [True, False]  # Whether bootstrap samples are used when building trees.
}

df21 =getdataRegress(21, True)
df22 =getdataRegress(22, True)
df23 = get_data(SETTELMENT,23)
df23 = remove_small_parties(df23,135720)
df23 = unite_parties(df23)
a=["HAIFA","ELAT","AYYELET HASHAHAR","SAKHNIN","QAZRIN"]
df23 = df23[df23.name.isin(a)]
locations=df23[['name']].values.tolist()
df23=df23[["committee code","Voters","Disqualified","Holders of voting rights","Likud",
           "Blue and white","Haredi","Right","Arab","Left","valid votes"]]

df=pd.concat([df21, df22])
x_train=df.iloc[:,:-1]
y_train=df.iloc[:,-1]
x_test=df23.iloc[:,:-1]
y_test=df23.iloc[:,-1]

estimator = RandomForestRegressor(random_state=SEED)

rs = RandomizedSearchCV(estimator, param_grid, n_jobs=-1,
                        scoring='neg_mean_squared_error', cv=5,
                        n_iter=1, verbose=1, random_state=SEED)

rs.fit(x_train,y_train)
predict_y = rs.predict(x_test)
actual=df23.iloc[:,-1].values
i=0
while i< len(predict_y):
        print("predection for "+str(locations[i])+ " is "+str(predict_y[i])+" while the true value is "+
                                    str(actual[i]))
        i=i+1
        


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    3.9s remaining:    5.9s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    4.0s finished
C:\Users\mzmir\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


predection for ['AYYELET HASHAHAR'] is 591.6584960317459 while the true value is 585
predection for ['ELAT'] is 24865.21497023809 while the true value is 22636
predection for ['HAIFA'] is 143864.38975992048 while the true value is 144625
predection for ['SAKHNIN'] is 15874.265541666677 while the true value is 17085
predection for ['QAZRIN'] is 3641.7413353174593 while the true value is 3498


In [15]:
from sklearn.ensemble import AdaBoostRegressor

df21=getdataRegress(21,False,1)
df22=getdataRegress(22,False,1)
df23 = get_data(SETTELMENT,23)
df23 = remove_small_parties(df23,135720)
df23 = unite_parties(df23)
a=["JERUSALEM","BENE BERAQ","SAKHNIN","KARMI'EL","DALIYAT AL-KARMEL"]
df23 = df23[df23.name.isin(a)]
locations=df23[['name']].values
df23=df23[["committee code","Voters","Disqualified","Holders of voting rights","Likud",
           "Haredi","Right","Arab","Left","valid votes","Blue and white"]]
print(df22)
df=pd.concat([df21, df22])
x_train=df.iloc[:,:-1]
y_train=df.iloc[:,-1]
x_test=df23.iloc[:,:-1]
y_test=df23.iloc[:,-1]
param_grid = {'n_estimators':[50,100,150,200,250], #Number of weak learners to train iteratively., 
                  'learning_rate':[0.001, 0.01, 0.1, 1], #It contributes to the weights of weak learners. It uses 1 as a default value.,
                   'loss' : ['linear', 'square', 'exponential']}

rs = RandomizedSearchCV(AdaBoostRegressor(),param_grid,cv=5,n_iter = 10,n_jobs=-1)
rs.fit(x_train, y_train)
ADB_best=rs.best_estimator_
adb = AdaBoostRegressor(ADB_best)
adb.fit(x_train, y_train)
predict_y = adb.predict(x_test)
actual=df23.iloc[:,-1].values
i=0
while i< len(predict_y):
        print("predection for "+str(locations[i])+ " is "+str(predict_y[i])+" while the true value is "+
                                    str(actual[i]))
        i=i+1


     committee code  Voters Disqualified Holders of voting rights  Likud  \
0                17     377            0                      517    128   
1                 2     245            0                      332     93   
2                17     287            0                      406     57   
3                18     373            0                      502     25   
4                17     228            1                      308    158   
...             ...     ...          ...                      ...    ...   
1209              8   21729          133                    34213   7385   
1210              8   20697          203                    36529   8626   
1211             19   34603           98                    45835   6954   
1212              9    7536           21                    10142   1404   
1213             99  282442         2226                        0  77953   

     Haredi  Right   Arab   Left valid votes Blue and white  
0        47     39      0

C:\Users\mzmir\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


predection for ['BENE BERAQ'] is 5020.254826254826 while the true value is 1133
predection for ['DALIYAT AL-KARMEL'] is 1454.009900990099 while the true value is 5200
predection for ['JERUSALEM'] is 32452.169851380044 while the true value is 32800
predection for ["KARMI'EL"] is 4377.3185840707965 while the true value is 6627
predection for ['SAKHNIN'] is 1454.009900990099 while the true value is 120


In [16]:
df21=getdataRegress(21,False,2)
df22=getdataRegress(22,False,2)
df23 = get_data(SETTELMENT,23)
df23 = remove_small_parties(df23,135720)
df23 = unite_parties(df23)
df23 = df23[df23.name.isin(a)]
locations=df23[['name']].values
df23=df23[["committee code","Voters","Disqualified","Holders of voting rights",
                   "Blue and white","Haredi","Right","Arab","Left","valid votes","Likud"]]
print(df22)
df=pd.concat([df21, df22])
x_train=df.iloc[:,:-1]
y_train=df.iloc[:,-1]
x_test=df23.iloc[:,:-1]
y_test=df23.iloc[:,-1]
param_grid = {'n_estimators':[50,100,150,200,250], #Number of weak learners to train iteratively., 
                  'learning_rate':[0.001, 0.01, 0.1, 1], #It contributes to the weights of weak learners. It uses 1 as a default value.,
                   'loss' : ['linear', 'square', 'exponential']}

rs = RandomizedSearchCV(AdaBoostRegressor(),param_grid,cv=5,n_iter = 10,n_jobs=-1)
rs.fit(x_train, y_train)
ADB_best=rs.best_estimator_
adb = AdaBoostRegressor(ADB_best)
adb.fit(x_train, y_train)
predict_y = adb.predict(x_test)
actual=df23.iloc[:,-1].values
i=0
while i< len(predict_y):
        print("predection for "+str(locations[i])+ " is "+str(predict_y[i])+" while the true value is "+
                                    str(actual[i]))
        i=i+1


     committee code  Voters Disqualified Holders of voting rights  \
0                17     377            0                      517   
1                 2     245            0                      332   
2                17     287            0                      406   
3                18     373            0                      502   
4                17     228            1                      308   
...             ...     ...          ...                      ...   
1209              8   21729          133                    34213   
1210              8   20697          203                    36529   
1211             19   34603           98                    45835   
1212              9    7536           21                    10142   
1213             99  282442         2226                        0   

     Blue and white Haredi  Right   Arab   Left valid votes  Likud  
0                98     47     39      0     44         377    128  
1                 6    108     11

C:\Users\mzmir\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


predection for ['BENE BERAQ'] is 5361.301282051282 while the true value is 4951
predection for ['DALIYAT AL-KARMEL'] is 441.69391304347823 while the true value is 407
predection for ['JERUSALEM'] is 54597.94594594595 while the true value is 72601
predection for ["KARMI'EL"] is 9922.970588235294 while the true value is 8879
predection for ['SAKHNIN'] is 534.407949790795 while the true value is 39
